# Topic Modeling of Reviews

## Data Preparation

First, we load the CSV, removing empty and very small text inputs to avoid simple things as "i like"

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("winemag-data_first150k.csv", nrows=10000)

df = df[["description"]]
df = df[(df["description"].str.strip() != "") & (df["description"].str.len() >= 40)]

print("Total: ", len(df["description"]))
print(df["description"][:5])

Total:  10000
0    This tremendous 100% varietal wine hails from ...
1    Ripe aromas of fig, blackberry and cassis are ...
2    Mac Watson honors the memory of a wine once ma...
3    This spent 20 months in 30% new French oak, an...
4    This is the top wine from La Bégude, named aft...
Name: description, dtype: object


We need to import nltk to find the english stopwords, which are helper words such as "from", "the", "that" that we should ignore to not compromise our result.

You need to uncomment the nltk download line and download the spanish corpus first.

In [2]:
import nltk

# uncomment if you need this
# nltk.download() 

stop_words = nltk.corpus.stopwords.words('english')

print(stop_words[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


Now we remove those stop words and banned words from the reviews because they would only mess with the algorithm, also we remove the accents because some reviews use it, some don't

In [3]:
import unidecode

# banned words, we don't want to cluster around those
banned_words = ['wine']

text = [" ".join([unidecode.unidecode(word) for word in text.split(" ")
                  if word not in stop_words
                  and unidecode.unidecode(word) not in banned_words
                 ]) for text in df["description"]]

text[:5]

['This tremendous 100% varietal hails Oakville aged three years oak. Juicy red-cherry fruit compelling hint caramel greet palate, framed elegant, fine tannins subtle minty tone background. Balanced rewarding start finish, years ahead develop nuance. Enjoy 2022-2030.',
 'Ripe aromas fig, blackberry cassis softened sweetened slathering oaky chocolate vanilla. This full, layered, intense cushioned palate, rich flavors chocolaty black fruits baking spices. A toasty, everlasting finish heady ideally balanced. Drink 2023.',
 'Mac Watson honors memory made mother tremendously delicious, balanced complex botrytised white. Dark gold color, layers toasted hazelnut, pear compote orange peel flavors, reveling succulence 122 g/L residual sugar.',
 "This spent 20 months 30% new French oak, incorporates fruit Ponzi's Aurora, Abetina Madrona vineyards, among others. Aromatic, dense toasty, deftly blends aromas flavors toast, cigar box, blackberry, black cherry, coffee graphite. Tannins polished fine s

Next we tokennize and stemmatized words so we can get their essential meaning and avoid considering different genders or verb inflections of a word as a distinct word when they are probably talking about the same thing.

Those words are saved in a vocabulary for later.

In [4]:
from nltk.stem.snowball import SnowballStemmer
import re

stemmer = SnowballStemmer("english")

def tokenize(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def stem(tokens):
    return [stemmer.stem(t) for t in tokens]

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in text:
    allwords_tokenized = tokenize(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
    allwords_stemmed = stem(allwords_tokenized)
    totalvocab_stemmed.extend(allwords_stemmed)
    

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')
print(vocab_frame.head())

there are 263552 items in vocab_frame
              words
this           this
tremend  tremendous
variet     varietal
hail          hails
oakvill    oakville


## Fitting the Model

Now, we are going to send our text to a TfidfVectorizer, where we use IDF to make the algorithm focus on the main no-so-common words. We also build ngrams, so things like "black cherry" can be considered a single thing instead of two.

Then we send it to a Latent Dirichlet Allocation model, which should group the texts around clusters, discovering topics automatically for us.

In [5]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import adjusted_rand_score


def tokenize_and_stem(text):
    return stem(tokenize(text))


vectorizer = TfidfVectorizer(max_df=0.6, max_features=200000,
                             min_df=5, stop_words=stop_words,
                             use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

X = vectorizer.fit_transform(text).todense()
print(X.shape)

terms = vectorizer.get_feature_names()
print(terms[90:100])

K = 7
model = LatentDirichletAllocation(n_components=K, max_iter=100, random_state=8)
model.fit(X)

model

(10000, 12207)
["'s medium", "'s medium bodi", "'s medium full", "'s medium-bodi", "'s miner", "'s much", "'s nice", "'s nice balanc", "'s nose", "'s nose palat"]


/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


CPU times: user 5min 38s, sys: 1.68 s, total: 5min 40s
Wall time: 5min 41s


## Visualization

Here we use pyLDAvis which give us an off-the-shelf visualization for LDA. The topics with more reviews in it are the biggest circles.

You can click in any of it to see which are the most relevant words for that particular topic, and you can move the relevace slider to the right to see words that are more relevant to that topic and not others.

This is a great visualization because we can see 3 things:
- The biggest topics that we have to concern about
- Which topics have overlaps and which talks about completely distinct subjects
- Which words are in those topics, giving us a hint of what it is about

In [7]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(model, X, vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.110336  0.060909       1        1  54.285219
5      0.220071 -0.083604       2        1  16.428643
2     -0.054400  0.216057       3        1  13.128428
6     -0.155405 -0.134441       4        1   7.992915
0     -0.052752 -0.028044       5        1   3.129000
1     -0.037270 -0.017232       6        1   2.665469
3     -0.030581 -0.013645       7        1   2.370326, topic_info=      Category        Freq                Term       Total  loglift  logprob
term                                                                        
3554   Default  166.000000               drink  166.000000  30.0000  30.0000
2080   Default   64.000000            cabernet   64.000000  29.0000  29.0000
7943   Default  185.000000               palat  185.000000  28.0000  28.0000
5090   Default  100.000000              fruiti  100.000000  27.0000  27.0000
1682   Default   89.000000               blend   89.000000  26.0000  26.0000
831    Default  183.000000               aroma  183.000000  25.0000  25.0000
4844   Default  219.000000               fruit  219.000000  24.0000  24.0000
2319   Default  150.000000              cherri  150.000000  23.0000  23.0000
400    Default   84.000000                 age   84.000000  22.0000  22.0000
546    Default   53.000000            alongsid   53.000000  21.0000  21.0000
11918  Default  135.000000                wine  135.000000  20.0000  20.0000
1615   Default   79.000000          blackberri   79.000000  19.0000  19.0000
7683   Default  106.000000               offer  106.000000  18.0000  18.0000
2257   Default   69.000000             charact   69.000000  17.0000  17.0000
9573   Default   49.000000           sauvignon   49.000000  16.0000  16.0000
9239   Default  145.000000                ripe  145.000000  15.0000  15.0000
2091   Default   39.000000  cabernet sauvignon   39.000000  14.0000  14.0000
6984   Default   41.000000              merlot   41.000000  13.0000  13.0000
9128   Default  109.000000                rich  109.000000  12.0000  12.0000
201    Default  172.000000                acid  172.000000  11.0000  11.0000
8074   Default   48.000000         palat offer   48.000000  10.0000  10.0000
1485   Default  144.000000               black  144.000000   9.0000   9.0000
11068  Default  106.000000              textur  106.000000   8.0000   8.0000
10504  Default   81.000000            structur   81.000000   7.0000   7.0000
10799  Default  155.000000              tannin  155.000000   6.0000   6.0000
8920   Default   51.000000               readi   51.000000   5.0000   5.0000
10703  Default   39.000000               syrah   39.000000   4.0000   4.0000
9224   Default   28.000000               riesl   28.000000   3.0000   3.0000
5314   Default   78.000000                give   78.000000   2.0000   2.0000
8922   Default   44.000000         readi drink   44.000000   1.0000   1.0000
...        ...         ...                 ...         ...      ...      ...
5138    Topic7    5.370509  fruiti readi drink    6.051093   3.6228  -5.4915
3042    Topic7    2.838157        crisp citrus    3.520647   3.5267  -6.1292
8520    Topic7    2.682525   plum black cherri    3.365527   3.5153  -6.1856
11570   Topic7    2.602885                verd    3.282442   3.5102  -6.2158
2796    Topic7    2.553352              common    3.237440   3.5048  -6.2350
11639   Topic7    2.473778          vinho verd    3.153115   3.4995  -6.2666
11638   Topic7    2.473778               vinho    3.153115   3.4995  -6.2666
10057   Topic7    2.389840          soft light    3.073101   3.4907  -6.3012
7055    Topic7    1.850682       miner element    2.534104   3.4279  -6.5568
5044    Topic7    1.809766  fruit spice flavor    2.491584   3.4224  -6.5792
6305    Topic7    1.678961      leather flavor    2.360206   3.4016  -6.6542
328     Topic7    1.666258          acid round    2.355451   3.3960  -6.6618
9783  

Of course we don't only want to see the chart about, we want to be able to effectively read the reviews in each topic, so, we build a dataframe with the original texts and the cluster number and score assigned to it. The score is the proximity of that review with the most revelant words in that cluster, so we filter those with less than 60% relevance because we only want the most focused ones.

In [8]:
result = model.transform(X)
clusters = [ np.argmax(score) for score in result ]
scores = [ max(score) for score in result ]

df["cluster"] = clusters
df["score"] = scores

df.loc[(df["score"] < 0.6), "cluster"] = K

df[["description","cluster","score"]].head()

,description,cluster,score
0,This tremendous 100% varietal wine hails from ...,4,0.872462
1,"Ripe aromas of fig, blackberry and cassis are ...",7,0.567521
2,Mac Watson honors the memory of a wine once ma...,4,0.822930
3,"This spent 20 months in 30% new French oak, an...",4,0.642081
4,"This is the top wine from La Bégude, named aft...",5,0.867355


Then we get the main words for each cluster, some examples and the sizes of the clusters

In [9]:
# copied from https://github.com/bmabey/pyLDAvis/blob/master/pyLDAvis/sklearn.py
def _get_topic_term_dists(lda_model):
    return _row_norm(lda_model.components_)

def _row_norm(dists):
    # row normalization function required
    # for doc_topic_dists and topic_term_dists
    return dists / dists.sum(axis=1)[:, None]

topic_term_dists = _get_topic_term_dists(model)

# sort cluster centers by proximity to centroid
order_centroids = topic_term_dists.argsort()[:, ::-1] 

cluster_words = [
    ", ".join([
        vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0] for ind in order_centroids[i, :6]
    ]) for i in range(K)
]
cluster_words.append("Others")

print(cluster_words[0])

cluster_examples = [
    df[df["cluster"] == i]["description"].values.tolist() for i in range(len(cluster_words))
]

print(cluster_examples[0][0])

cluster_sizes = [ len(df[df["cluster"] == i]) for i in range(len(cluster_words)) ]

print(cluster_sizes)

rubbery, flavors, raspberry, resiny, saucy, plum


IndexError: list index out of range

And we print everything for you to read

In [10]:
data = { 'text': df["text"].values.tolist() }

frame = pd.DataFrame(data, index = [clusters], columns = ['text'])

print("Top terms per cluster:")
print()

for i in range(len(cluster_sizes)):
    print("Cluster %d total:" % i, cluster_sizes[i])
    print() #add whitespace
    
    print(cluster_words[i])
    print() #add whitespace
    
    print("Cluster %d examples:" % i)
    for title in cluster_examples[i][:15]:
        print('- %s' % title)
    print() #add whitespace
    print() #add whitespace

KeyError: 'text'

So, one problem with Unsupervised Learning of clustering is that you have to choose beforehand the number of clusters that you want, previously, we choose a small number, and maybe creating topics that are actually talking about two different things.

On the other hand, if we choose too many topics we might break those topics in smaller ones, but create many one which are actually talking about the same things.

We can visualize this effect on the chart, take a look with K = 30

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
K2 = 30

model2 = LatentDirichletAllocation(n_components=K2, max_iter=100, random_state=0)
model2.fit(X)

pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(model2, X, vectorizer)

## Credits

I only managed to create this notebook thanks to those two awesome tutorials, which I basically copied for this topic modelling work:

1. Document Clustering with Python - http://brandonrose.org/clustering
2. Modern NLP in Python - https://www.youtube.com/watch?v=6zm9NC9uRkk

And thanks for the [@datasciencepython](https://web.telegram.org/#/im?p=@datasciencepython) group in telegram for pointing me in the right direction